In [4]:
from hyp3_sdk import HyP3
import asf_search as asf
from config import hyp3_username, hyp3_password

            

In [ ]:
hyp3 = HyP3(username=hyp3_username, password=hyp3_password)
cali_poly = 